In [18]:
import pandas as pd
import sys
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import re
import itertools
import datetime as dt
import sqlite3

# connect to folder for custom functions
root = Path.cwd().parent
vis_path = root / "src" / "visualizations" 
mod_path = root / "src" / "modelling" 
sys.path.append(str(vis_path))
sys.path.append(str(mod_path))

from read_articles import read
from plot_functions import plot_aini_series_subplots
from compute_extrema import compute_aini_extrema

In [19]:
# prepare paths
var_path = root / "data" / "processed" / "variables"
art_path = root / "data" / "processed" / "articles"
table_path = root / "reports" / "tables"

# load aini data
aini_custom = pd.read_csv(var_path / "binary_AINI_variables.csv")
aini_w0 = pd.read_csv(var_path / "w0_AINI_variables.csv")
aini_w1 = pd.read_csv(var_path / "w1_AINI_variables.csv")
aini_w2 =  pd.read_csv(var_path / "w2_AINI_variables.csv")


In [20]:
# create dataframes for visualizaions
normalized_AINI = pd.DataFrame()

# ensure sorting
aini_w0 = aini_w0.sort_values("date")
aini_w1 = aini_w1.sort_values("date")
aini_w2 = aini_w2.sort_values("date")
aini_custom = aini_custom.sort_values("date")

# compute extrema
merged, tidy, pivot, extrema = compute_aini_extrema(aini_w0,aini_w1,aini_w2,aini_custom)
merged

Explore differences in min, max, mean and std.

In [21]:
def summarize_columns(df, exclude=["date"]):
    """Return mean, std, min, max for each numeric column in df (except exclude)."""
    results = []
    for col in df.columns:
        if col not in exclude:
            series = df[col]
            results.append({
                "variable": col,
                "mean": series.mean(),
                "std": series.std(),
                "min": series.min(),
                "max": series.max()
            })
    return pd.DataFrame(results)

# usage
stats_individual = summarize_columns(merged)
print(stats_individual)

In [22]:
# write to csv
extrema.to_csv(table_path / "aini_extrema.csv")


In [ ]:
# Convert to LaTeX with booktabs, tabular (single-page), wrapped in adjustbox
latex_table = extrema.to_latex(
    index=False,
    escape=True,
    column_format="l" + "c" * (len(extrema.columns) - 1),
    bold_rows=False
)

# Add booktabs spacing
latex_table = latex_table.replace("\\toprule", "\\toprule\n\\addlinespace")
latex_table = latex_table.replace("\\midrule", "\\midrule\n\\addlinespace")
latex_table = latex_table.replace("\\bottomrule", "\\addlinespace\n\\bottomrule")

# Wrap in table + adjustbox
latex_wrapped = (
    "\\begin{table}[!htbp]\n"
    "\\centering\n"
    "\\begin{adjustbox}{width=\\textwidth}\n"
    + latex_table +
    "\\end{adjustbox}\n"
    "\\caption{AINI extrema}\n"
    "\\label{tab:aini_extrema}\n"
    "\\end{table}\n"
)

# Save to file
output_path = table_path / "aini_extrema.tex"
with open(output_path, "w") as f:
    f.write(latex_wrapped)

Calculate weekly extrema (by calendar week)

In [ ]:

# derive calendar week (ISO year + week number)
tidy["week"] = tidy["date"].dt.to_period("W").apply(lambda r: r.start_time)

# Count how often each min/max week occurs
counts_by_week = (
    tidy.groupby(["type", "week"])
        .size()
        .reset_index(name="count")
        .sort_values(["type", "count"], ascending=[True, False])
)

# subset n > 0
extrema_weekly = counts_by_week[counts_by_week["count"] > 0]

# collect variables for each week
week_dict = (
    tidy.groupby(["week"])["variable"]
    .apply(list)
    .to_dict()
)

# attach variables to each week
extrema_weekly["measure"] = extrema_weekly["week"].map(week_dict)

# bring into convenient format
extrema_weekly_clean = extrema_weekly.copy()

# week start (Monday)
week_start = extrema_weekly_clean["week"]
# week end (Sunday) = start + 6 days
week_end = week_start + pd.Timedelta(days=6)

# format as "dd.mm.yyyy - dd.mm.yyyy"
extrema_weekly_clean["week"] = (
    week_start.dt.strftime("%d.%m.%Y") + " - " + week_end.dt.strftime("%d.%m.%Y")
)

extrema_weekly_clean["type"] = extrema_weekly_clean["type"].replace({"min": "minimum", "max": "maximum"})
extrema_weekly_clean.rename(columns={"count": "n measures"}, inplace=True)

# save
extrema_weekly_clean.to_csv(table_path / "aini_weekly_extrema.csv", index=False)

In [ ]:
# Convert to LaTeX with booktabs, tabular (single-page), wrapped in adjustbox
latex_table = extrema_weekly_clean.to_latex(
    index=False,
    escape=True,
    column_format="l" + "c" * (len(extrema_weekly_clean.columns) - 1),
    bold_rows=False
)

# Add booktabs spacing
latex_table = latex_table.replace("\\toprule", "\\toprule\n\\addlinespace")
latex_table = latex_table.replace("\\midrule", "\\midrule\n\\addlinespace")
latex_table = latex_table.replace("\\bottomrule", "\\addlinespace\n\\bottomrule")

# Wrap in table + adjustbox
latex_wrapped = (
    "\\begin{table}[!htbp]\n"
    "\\centering\n"
    "\\begin{adjustbox}{width=\\textwidth}\n"
    + latex_table +
    "\\end{adjustbox}\n"
    "\\caption{AINI extrema}\n"
    "\\label{tab:aini_extrema}\n"
    "\\end{table}\n"
)

# Save to file
output_path = table_path / "aini_extrema_weekly.tex"
with open(output_path, "w") as f:
    f.write(latex_wrapped)

In [ ]:
extrema.sort_values("n measures",ascending=False)

In [ ]:
extrema_weekly_clean.sort_values("n measures",ascending=False)

Note: Min1 & Min2 in week with most Minima; 
Max1 is in week with most maxima, Max2 in second week with most maxima. 

load data with AINI predictions & compare with maxima and minima

In [ ]:
# load & merge w1 data custom finbert data
c_df23 = pd.read_csv(var_path / "FinBERT_AINI_prediction_2023_on_binary.csv")
c_df24 = pd.read_csv(var_path / "FinBERT_AINI_prediction_2024_on_binary.csv")
c_df25 = pd.read_csv(var_path / "FinBERT_AINI_prediction_2025_on_binary.csv")
c_df = pd.concat([c_df23,c_df24,c_df25]) 

# load & merge w0 data
w0_df23 = pd.read_csv(var_path / "FinBERT_AINI_prediction_2023_windsize_0.csv")
w0_df24 = pd.read_csv(var_path / "FinBERT_AINI_prediction_2024_windsize_0.csv")
w0_df25 = pd.read_csv(var_path / "FinBERT_AINI_prediction_2025_windsize_0.csv")
w0_df = pd.concat([w0_df23,w0_df24,w0_df25]) 

# load & merge w1 data
w1_df23 = pd.read_csv(var_path / "FinBERT_AINI_prediction_2023_windsize_1.csv")
w1_df24 = pd.read_csv(var_path / "FinBERT_AINI_prediction_2024_windsize_1.csv")
w1_df25 = pd.read_csv(var_path / "FinBERT_AINI_prediction_2025_windsize_1.csv")
w1_df = pd.concat([w1_df23,w1_df24,w1_df25]) 

# merge on normalized_aini_wo to identify relevant articles
w2_df23 = pd.read_csv(var_path / "FinBERT_AINI_prediction_2023_windsize_2.csv")
w2_df24 = pd.read_csv(var_path / "FinBERT_AINI_prediction_2024_windsize_2.csv")
w2_df25 = pd.read_csv(var_path / "FinBERT_AINI_prediction_2025_windsize_2.csv")
w2_df = pd.concat([w2_df23,w2_df24,w2_df25]) 

# create df list 
aini_dfs = [c_df,w0_df,w1_df,w2_df]

In [ ]:
# verify integrity
for df in aini_dfs:
    print(
        f"First entry: {df.date.min()} "
        f"\n last entry: {df.date.max()}, "
        f"\n n entries = {len(df)}, "
        f"\n n non-unique ids: {df['article_id'].duplicated().sum()}",
        f"Columns: {df.columns}"
    )

In [ ]:
# subset for relevant columns
rel_col = ["article_id","sentiment_label","sentiment_score","hype_score"]
clean_df = []

for df in aini_dfs:
    df = df[rel_col].copy()
    clean_df.append(df)
    
c_df_sub,w0_df_sub,w1_df_sub,w2_df_sub = clean_df
c_df_sub

In [ ]:
# list of DB paths
db_files = [
    art_path / "articlesWSJ_clean_2023.db",
    art_path / "articlesWSJ_clean_2024.db",
    art_path / "articlesWSJ_clean_2025.db",
]

dfs = []
for db in db_files:
    with sqlite3.connect(db) as conn:

        # read data base into data frame
        df = pd.read_sql("SELECT * FROM article", conn)
        dfs.append(df)

# combine into one DataFrame
all_articles = pd.concat(dfs, ignore_index=True)

# verify integrity
print(
    f"First entry: {all_articles.date.min()} "
    f"\n last entry: {all_articles.date.max()}, "
    f"\n n entries = {len(all_articles)}, "
    f"\n n non-unique ids: {all_articles['article_id'].duplicated().sum()}",
    f"Columns: {all_articles.columns}"
)

In [ ]:
# harmonize key dtype
all_articles["article_id"] = all_articles["article_id"].astype(str)
c_df_sub["article_id"]  = c_df_sub["article_id"].astype(str)
w0_df_sub["article_id"] = w0_df_sub["article_id"].astype(str)
w1_df_sub["article_id"] = w1_df_sub["article_id"].astype(str)
w2_df_sub["article_id"] = w2_df_sub["article_id"].astype(str)

# start from base
complete_df = all_articles.copy()

# merge step by step with suffixes
complete_df = complete_df.merge(c_df_sub,  on="article_id", how="left", suffixes=("", "_c"))
complete_df = complete_df.merge(w0_df_sub, on="article_id", how="left", suffixes=("", "_w0"))
complete_df = complete_df.merge(w1_df_sub, on="article_id", how="left", suffixes=("", "_w1"))
complete_df = complete_df.merge(w2_df_sub, on="article_id", how="left", suffixes=("", "_w2"))


# verify integrity
print(
    f"First entry: {complete_df.date.min()} "
    f"\n last entry: {complete_df.date.max()}, "
    f"\n n entries = {len(complete_df)}, "
    f"\n n non-unique ids: {complete_df['article_id'].duplicated().sum()}",
    f"Columns: {w0_df.columns}"
)

In [ ]:
# ensure format in date col 
complete_df["dates"] = pd.to_datetime(complete_df["date"])

# define minima
min_1 = pd.Timestamp("2025-02-06") # n=8
min_2 = pd.Timestamp("2025-02-04") # n=2

# define maxima
max_1 = pd.Timestamp("2025-06-16") # n=6
max_2 = pd.Timestamp("2025-04-01")  # n=4

# subset original data extrema, minima
articles_min1 = complete_df[complete_df["dates"] == min_1] 
articles_min2 = complete_df[complete_df["dates"] == min_2]

# subset original data extrema, maxmima
articles_max1 = complete_df[complete_df["dates"] == max_1] 
articles_max2 = complete_df[complete_df["dates"] == max_2] 

# investigate structure
articles_max1

In [ ]:
# investigate 06.02.2025; min 1
read(articles_min1)

In [ ]:
# investigate 16.06.2025; max 1 according to ['normalized_AINI_w1', 'normalized_AINI_w2', 'EMA_02_w1', 'EMA_02_w2', 'EMA_08_w1', 'EMA_08_w2']
articles_max1

In [ ]:
# investigate 01.04.2023; max 2 according to normalized_AINI_custom, simple_AINI_custom, EMA_02_custom, EMA_08_custom
articles_max2

In [ ]:
# ensure datetime type
complete_df["date"] = pd.to_datetime(complete_df["date"], errors="coerce")

# subsets per year
df_2023 = complete_df[complete_df["date"].dt.year == 2023]
df_2024 = complete_df[complete_df["date"].dt.year == 2024]
df_2025 = complete_df[complete_df["date"].dt.year == 2025]

Investigate AINI by year

In [ ]:
plot_aini_series_subplots(merged)